# このページには、2010年以降のアカハニュースについて簡単に統計を取ったものが表示されています。

以下のページのにある表``【日本の研究者の性不正・アカハラ事件一覧】``を元に簡単な統計処理をしたものです。

：[研究者のセクハラ・アカハラ事件一覧 | 研究者倫理]((https://haklak.com/page_harassment_2.html))

---
# 最初のこの部分はデータ処理をしている所なので読み飛ばしてください。

**スクレイピング**

このページの表``【日本の研究者の性不正・アカハラ事件一覧】``を抽出

[研究者のセクハラ・アカハラ事件一覧 | 研究者倫理](https://haklak.com/page_harassment_2.html)


2010年以降の部分だけ（2010年以前は一部だけみたいなので）

In [32]:
import pandas as pd
import numpy as np
import os
from glob import glob

import requests
from bs4 import BeautifulSoup

In [33]:
url = f'https://haklak.com/page_harassment_2.html'

# スクレイピング
# urllib.request だと 503 service unavailable が出たので requests.getで
page = requests.get(url)

# パース
soup = BeautifulSoup(page.text, 'html.parser')

In [34]:
# tableを取得
table = soup.find_all("table")[1]

# 各行を取得
rows = table.find_all('tr')

# ここに追加していく
values_ary = []
columns = []

# セルの値を追加していく
for i, row in enumerate(rows):
    
    # 一行目はheader
    if i == 0:
        for cell in row.find_all('th'):
            v = cell.get_text()
            columns.append(v)

    # 2行目以降はcellの値
    else:    
        sub_ary = []
        for cell in row.find_all('td'):
            v = cell.get_text()
            sub_ary.append(v)
        values_ary.append(sub_ary)

df = pd.DataFrame(values_ary, columns=columns)

# 2010年以降を抽出
df = df.iloc[:179, :]

表の最初の2行を表示（どんな表か見るため）

In [35]:
display(df.head(2))

,公表年月日,名前(姓 名) SPACE,性,事件種,被害者 SPACESP,行為内容（文章は出典から修正引用） SPACESPACESPACESPACE,所属機関 SPAC,領域,地位,公表時年齢,職,処分など SPAC,透明性,出典
0,191113,匿名,男,レイプ,20代の女性卒業生1人,酩酊して眠っている状態のときに、同意なく性行為を強いられた,福島大学,？,教授、学類長,50代,✕,なし、自己都合退職,✕,191113田中圭太郎 191114大学
1,191018,匿名,男,セク、ストーキング,女性学生,頭や肩などに触れたり、学会出張時の列車内でひざを触った,山形大学,工,准教授,50代,〇,停職1か月,✕,191018産経


column名表示（後でコピペするため）

In [36]:
print(df.columns.tolist())

['公表年月日', '名前(姓 名) SPACE', '性', '事件種', '被害者 SPACESP', '行為内容（文章は出典から修正引用） SPACESPACESPACESPACE', '所属機関 SPAC', '領域', '地位', '公表時年齢', '職', '処分など SPAC', '透明性', '出典']


# ここまでがデータ処理の部分です。
---
# ここからが統計処理をした後のものです。

# ・表の簡単な統計値を表示

In [37]:
df.describe()

,公表年月日,名前(姓 名) SPACE,性,事件種,被害者 SPACESP,行為内容（文章は出典から修正引用） SPACESPACESPACESPACE,所属機関 SPAC,領域,地位,公表時年齢,職,処分など SPAC,透明性,出典
count,179,179,179,179,179,179,179,179,179,179,179,179,179,179
unique,169,41,5,27,134,168,103,38,26,35,10,103,7,164
top,190329,匿名,男,アカ,,２０１０～１２年、,山形大学,？,教授,50代,〇,？,✕,130715北海道
freq,2,139,154,62,6,5,7,43,95,49,127,11,149,5


# ・いくつかの項目について、種類とその出現数を表示

In [48]:
def display_value_counts(name):    
    dff = df[name].value_counts().to_frame()
    dff.columns = ['出現数']
    dff.index.name = '種類'
    dff = dff.reset_index()
    display(dff)

### 性

In [49]:
display_value_counts('性')

,種類,出現数
0,男,154
1,？,11
2,女,9
3,ー,3
4,,2


### 事件種

In [50]:
display_value_counts('事件種')

,種類,出現数
0,アカ,62
1,セク,29
2,性的暴行,22
3,セク、アカ,18
4,ハラ,10
5,アカ、セク,6
6,レイプ,5
7,性的暴行、アカ,3
8,強制わいせつ,3
9,アカ？,2


### 所属機関 SPAC

In [51]:
display_value_counts('所属機関 SPAC')

,種類,出現数
0,山形大学,7
1,北海道大学,6
2,早稲田大学,5
3,広島大学,4
4,九州工業大学,4
5,九州大学,4
6,金沢大学,4
7,大阪市立大学,4
8,島根大学,4
9,大阪大学,4


### 地位

In [52]:
display_value_counts('地位')

,種類,出現数
0,教授,95
1,准教授,33
2,教員,8
3,学長,6
4,助教,5
5,教授ら５人,5
6,講師,5
7,医師,2
8,学部長,2
9,ー,2


### 公表時年齢

In [53]:
display_value_counts('公表時年齢')

,種類,出現数
0,50代,49
1,？,41
2,40代,22
3,60代,9
4,30代,7
5,58,4
6,63,3
7,61,3
8,ー,3
9,64,3


# ・公表時年齢 を元に 世代別件数 を計算

In [63]:
# ごみを削除
tmp = df['公表時年齢'].replace({'代': '', '歳': '', '\?': '', '\？': '', 'ー': ''}, regex=True)

# nan はいったん -100 にしておく
tmp = tmp.replace('', '-100')

# 値を世代に変更
tmp = tmp.map(lambda x: int(x) // 10 * 10)

# nanを戻す
tmp = tmp.replace(-100, '不明')

# dfに
tmp = tmp.value_counts().to_frame()

# 表示を整形
tmp.columns = ['件数']
tmp.index.name = '世代'
tmp = tmp.reset_index()

# 表示
tmp

,世代,件数
0,50,62
1,不明,49
2,60,31
3,40,26
4,30,10
5,80,1
